## Table Of Contents
1. [Problem_Statement](#ps)
2. [Environment](#env)
2. [Observations](#obs)
3. [Declaring random and SlateQ agents](#create)
4. [Training and evaluating agents](#train)
5. [Results](#result)

Inspired by [this](https://github.com/google-research/recsim/blob/master/recsim/colab/RecSim_Overview.ipynb) tutorial.

In [15]:
import numpy as np
import tensorflow as tf

from recsim.environments import interest_evolution
from recsim.agents import full_slate_q_agent, random_agent
from recsim.simulator import runner_lib

<a id='ps'></a>

## Problem Statement

Most practical recommender systems focus on estimating immediate user engagement without considering the long-term effects of recommendations on user behaviour. Reinforcement learning (RL) methods offer the potential to optimize recommendations for long-term user engagement. However, since users are often presented with slates of multiple items—which may have interacting effects on user choice—methods are required to deal with the combinatorics of the RL action space.

Google’s SlateQ algorithm addresses this challenge by decomposing the long-term value (LTV) of a slate of items into a tractable function of its component item-wise LTVs. For our project, we want to compare the efficiency of SlateQ to other RL methods like Deep Q-Networks that don’t decompose the LTV of a slate into its component-wise LTVs.

<a id='env'></a>

## Environment

The `Interest Evolution` environment depicts a situation in which a user of an online service interacts with items of content,  which are characterized by their a vector of video features along with its quality.

Each user starts with a fixed initial budget of time, which decreases with each document consumed. However, the budget is also replenished depending upon the percieved interest of the user in the shown document.

The challenge is to train an agent that can recommend items basis evolving user behavior and interests.

![title](../images/simulator.png)

<a id='obs'></a>

## Observations

A RecSim observation is a dictionary with 3 keys:

- 'user', which represent the 'User Observable Features' in the structure diagram above,
- 'doc', containing the current corpus of recommendable videos and their observable features ('Document Observable Features'),
- and 'response', indicating the user's response to the last slate of recommendations ('User Response'). At this stage the 'response' key is vacuous and will be set to None, as no recommendation has been made yet.

In [16]:
# setting environment variables and sampling

seed = 0
np.random.seed(seed)
env_config = {
  'num_candidates': 10,  # number of videos to choose from
  'slate_size': 2,
  'resample_documents': True,
  'seed': seed,
  }
ie_environment = interest_evolution.create_environment(env_config)
initial_observation = ie_environment.reset()

In [17]:
# print sample record

print('User Observable Features')
print(initial_observation['user'])
print('User Response')
print(initial_observation['response'])
print('Document Observable Features')
for doc_id, doc_features in initial_observation['doc'].items():
  print('ID:', doc_id, 'features:', doc_features)

User Observable Features
[ 0.95723668  0.59831713 -0.07704128  0.56105835 -0.76345115  0.27984204
 -0.71329343  0.88933783  0.04369664 -0.17067612 -0.47088878  0.54846738
 -0.08769934  0.1368679  -0.9624204   0.23527099  0.22419145  0.23386799
  0.88749616  0.3636406 ]
User Response
None
Document Observable Features
ID: 10 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
ID: 11 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
ID: 12 features: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ID: 13 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
ID: 14 features: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ID: 15 features: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ID: 16 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ID: 17 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
ID: 18 features: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

Documents space consists of a slate of 10 items where each document is  represented by a 20 dimensional vector. Each element is a float in range [-1,1].

In [18]:
# print sample record

print('Document observation space')
for key, space in ie_environment.observation_space['doc'].spaces.items():
  print(key, ':', space)

Document observation space
10 : Box(20,)
11 : Box(20,)
12 : Box(20,)
13 : Box(20,)
14 : Box(20,)
15 : Box(20,)
16 : Box(20,)
17 : Box(20,)
18 : Box(20,)
19 : Box(20,)


User Response has following elements:

- Click: A boolean indicating whether the video was clicked
- cluster_id: an integer for the cluster ID of the document, out of a total of 20 Topics. One document belongs only to one topic.
- watch_time: A float for fraction of the video watched. Lies between 0 and 100. 
- liked: A boolean indicating whether the video was liked
- quality: A float for document quality. Lies in range [-100,100]

In [19]:
# print sample response

print('Response observation space')
print(ie_environment.observation_space['response'])

Response observation space
Tuple(Dict(click:Discrete(2), cluster_id:Discrete(20), liked:Discrete(2), quality:Box(), watch_time:Box()), Dict(click:Discrete(2), cluster_id:Discrete(20), liked:Discrete(2), quality:Box(), watch_time:Box()))


A User can observe the document features discussed above.

In [20]:
# print sample record

print('User observation space')
print(ie_environment.observation_space['user'])

User observation space
Box(20,)


Our RecSim slate is a list of first $2$ indices of obeservation['doc']. E.g. the slate [0, 1] corresponds to the slate consisting of:

In [21]:
slate = [0, 1]
for slate_doc in slate:
  print(list(initial_observation['doc'].items())[slate_doc])

('10', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0.]))
('11', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0.]))


In [22]:
ie_environment.action_space

MultiDiscrete([10 10])

In [23]:
# print sample episode with rewards

observation, reward, done, _ = ie_environment.step(slate)
print(observation)
print(reward)
print(done)

{'user': array([ 0.95723668,  0.59831713, -0.07704128,  0.56105835, -0.76345115,
        0.27984204, -0.71329343,  0.88933783,  0.04369664, -0.17067612,
       -0.47088878,  0.54846738, -0.08769934,  0.1368679 , -0.9624204 ,
        0.05982786,  0.22419145,  0.23386799,  0.88749616,  0.3636406 ]), 'doc': OrderedDict([('20', array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])), ('21', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0.])), ('22', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0.])), ('23', array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])), ('24', array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])), ('25', array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0.])), ('26', array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

<a id='create'></a>

## Creating Agents

In [24]:
# creating agents

def create_q_agent(sess, environment, eval_mode, summary_writer=None):
  kwargs = {
      'observation_space': environment.observation_space,
      'action_space': environment.action_space,
      'summary_writer': summary_writer,
      'eval_mode': eval_mode,
  }
  return full_slate_q_agent.FullSlateQAgent(sess, **kwargs)


def create_random_agent(sess, environment, eval_mode, summary_writer=None):
    kwargs = {
        'action_space': environment.action_space
  }
    return random_agent.RandomAgent(**kwargs)

<a id='train'></a>

## Training the Agents

### Configuring the Environment

In [25]:
# environment config

seed = 0
np.random.seed(seed)
env_config = {
  'num_candidates': 10,
  'slate_size': 2,
  'resample_documents': True,
  'seed': seed,
  }

### Training the Random Agent

In [26]:
# training

tmp_random_dir = './results/baseline_random/'
runner = runner_lib.TrainRunner(
    base_dir=tmp_random_dir,
    create_agent_fn=create_random_agent,
    env=interest_evolution.create_environment(env_config),
    episode_log_file="",
    max_training_steps=50,
    num_iterations=10)
runner.run_experiment()

INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:Beginning training...


INFO:tensorflow:Beginning training...


INFO:tensorflow:Reloaded checkpoint and will start from iteration 10


INFO:tensorflow:Reloaded checkpoint and will start from iteration 10


### Training the Standard Full Slate Q-Agent

In [27]:
# training

tmp_q_dir = './results/baseline_fullq/'
runner = runner_lib.TrainRunner(
    base_dir=tmp_q_dir,
    create_agent_fn=create_q_agent,
    env=interest_evolution.create_environment(env_config),
    episode_log_file="",
    max_training_steps=50,
    num_iterations=10)
runner.run_experiment()

INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_training_steps = 50, number_iterations = 10,checkpoint frequency = 1 iterations.


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:Creating FullSlateQAgent agent with the following parameters:


...

INFO:tensorflow:Beginning training...


INFO:tensorflow:Restoring parameters from ./results/baseline_fullq/train/checkpoints/tf_ckpt-9


INFO:tensorflow:Restoring parameters from ./results/baseline_fullq/train/checkpoints/tf_ckpt-9


INFO:tensorflow:Reloaded checkpoint and will start from iteration 10


INFO:tensorflow:Reloaded checkpoint and will start from iteration 10


### Evaluating the Random Agent

In [28]:
# evaluating

runner = runner_lib.EvalRunner(
      base_dir=tmp_random_dir,
      create_agent_fn=create_random_agent,
      env=interest_evolution.create_environment(env_config),
      max_eval_episodes=5,
      test_mode=True)
runner.run_experiment()

INFO:tensorflow:max_eval_episodes = 10


INFO:tensorflow:max_eval_episodes = 10


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:eval_file: ./results/baseline_random/eval_10/returns_600


INFO:tensorflow:eval_file: ./results/baseline_random/eval_10/returns_600


### Evaluating the Q Agent

In [29]:
# evaluating

runner = runner_lib.EvalRunner(
      base_dir=tmp_q_dir,
      create_agent_fn=create_q_agent,
      env=interest_evolution.create_environment(env_config),
      max_eval_episodes=5,
      test_mode=True)
runner.run_experiment()

INFO:tensorflow:max_eval_episodes = 10


INFO:tensorflow:max_eval_episodes = 10


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:max_steps_per_episode = 27000


INFO:tensorflow:Creating FullSlateQAgent agent with the following parameters:


...

INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:Beginning evaluation...


INFO:tensorflow:Restoring parameters from ./results/baseline_fullq/train/checkpoints/tf_ckpt-9


INFO:tensorflow:Restoring parameters from ./results/baseline_fullq/train/checkpoints/tf_ckpt-9


ValueError: could not broadcast input array from shape (11,20) into shape (11,1)

<a id='result'></a>

## Viewing Results

In [30]:
%load_ext tensorboard

### Random Agent Results

In [31]:
%tensorboard --logdir=./results/baseline_random/

Reusing TensorBoard on port 6015 (pid 4260), started 15:27:06 ago. (Use '!kill 4260' to kill it.)

### Q Agent Results

In [32]:
%reload_ext tensorboard

In [33]:
%tensorboard --logdir=./results/baseline_fullq/ --port=8008

Reusing TensorBoard on port 8008 (pid 4411), started 15:24:53 ago. (Use '!kill 4411' to kill it.)